In [2]:
from azureml.core import Workspace, Experiment, Environment, Dataset
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
import json

In [3]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [4]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Prepare Script

In [5]:
%%writefile ./src/prepare.py

from azureml.core import Run
from argparse import ArgumentParser
import numpy as np
import pandas as pd
import datetime
import os

run = Run.get_context()

parser = ArgumentParser()
parser.add_argument('--output', dest='prepared_data')
args = parser.parse_args()

# load datasets
df_symptoms = run.input_datasets['symptomcodes'].to_pandas_dataframe()
df = run.input_datasets['df_raw'].to_pandas_dataframe()

###########################################################

# get only data from last t years
t = 5
df = df[df['Job Card.Date Start Work']>(datetime.datetime.today() - datetime.timedelta(days=t*365))]

############################################################

# clean data
df = df.replace(['', '0', '-', '000','N/A'], np.nan)
df = df.dropna().reset_index(drop=True)

#############################################################################

# combine Component/Failure Code in train data
df = pd.concat([df, pd.DataFrame(df.apply(lambda x: (x['Job Card.ComponentCode'],x['Job Card.FailureCode']), axis=1), columns=['CompFail'])], axis=1)

# combine Component/Failure Code in symptom table
df_symptoms = df_symptoms[['ComponentCode', 'FailureCode', 'Symptom1', 'Symptom2', 'Symptom3', 'Symptom4']]
df_symptoms = pd.concat([df_symptoms, pd.DataFrame(df_symptoms.apply(lambda x: (x['ComponentCode'],x['FailureCode']),axis=1), columns=['CompFail'])],axis=1)

# merge train data on symptoms
df = pd.merge(df, df_symptoms, on='CompFail', how='left')
df = pd.concat([df, pd.DataFrame(df[['Symptom1', 'Symptom2', 'Symptom3', 'Symptom4']].apply(lambda x: tuple([ x[col] for col in ['Symptom1','Symptom2','Symptom3','Symptom4'] if str(x[col]) != 'None' ]), axis=1), columns=['Symptoms'])], axis=1)

##############################################################################

# merge into one row per case
df = df.groupby('Job Card.JobCard Number').apply(lambda x: pd.Series({
    'ProductId': ' '.join(x['Installed Base.InstalledBase ProductID'].unique())
    'ProductNr': ' '.join(x['Product.Product Number'].unique()),
    'Symptoms': ' '.join(map(str, list(set(x['Symptoms'].sum())))),
    'Start': x['Job Card.Date Start Work'].min(),
    'End': x['Job Card.Date End Work'].max()
  })).reset_index()

##############################################################################

# compute duration column
df = pd.concat([df, pd.DataFrame((df['End'] - df['Start']), columns=['duration'])],axis=1)
df['duration'] = df['duration'].apply(lambda x: x.seconds / 3600)

##############################################################################

# save train and test data
path = arg.prepared_data if args.prepared_data else './outputs'
os.makedirs(path, exist_ok=True)
df.to_csv(path + '/prepared_data.csv', sep=';', header=True, index=False)

run.complete()

Overwriting ./src/prepare.py


## Create Estimator

In [6]:
est = Estimator(entry_script='prepare.py', source_directory='src', 
              inputs=[   ws.datasets[cfg['symptomcodes_dataset']].as_named_input('symptomcodes'), 
                         ws.datasets[cfg['raw_data_dataset']].as_named_input('df_raw')       ],
              compute_target=ws.compute_targets['mlcompute'], environment_definition=ws.environments[cfg['env_name']])

## Run Experiment

In [7]:
exp = Experiment(ws, cfg['experiment_name'])
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: SKLearnPrediction_1592504466_009bf69b
Web View: https://ml.azure.com/experiments/SKLearnPrediction/runs/SKLearnPrediction_1592504466_009bf69b?wsid=/subscriptions/793146d9-d4dc-4a73-9728-76c4ffd0cc0d/resourcegroups/rg_dynamics_test/workspaces/resdynml1test


ERROR - Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/core/run.py", line 681, in wait_for_completion
    raise_on_error=raise_on_error)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/core/run.py", line 868, in _stream_run_output
    time.sleep(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-c06bceba7052>", line 3, in <module>
    run.wait_for_completion(show_output=True)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/core/run.py", line 689, in wait_for_completion
    raise ExperimentExecutionException(error_message)
azureml.exceptions._azureml_exception.ExperimentExecutionException: ExperimentExecutionException

TypeError: must be str, not list

## Register Datasets

In [12]:
run.download_file('outputs/prepared_data.csv', output_file_path='artifacts/prepared_data.csv')
ds = ws.datastores[cfg['storage']]
data_ref = ds.upload_files(['artifacts/prepared_data.csv'], target_path='./'+cfg['experiment_name'], overwrite=True)
prepared_data_dataset = Dataset.Tabular.from_delimited_files(data_ref, separator=';', header=True, infer_column_types=True)
prepared_data_dataset.register(ws, cfg['prepared_data_dataset'], create_new_version=True)